In [1]:
import os, sys
sys.path.append("/home/chendian/CDNER")

In [3]:
import json

data_dir = '/home/chendian/CDNER/data'
en_dataset_names = [
    'conll03', 'ace04', 'ace05', 'scierc',
]

cn_dataset_names = [
    'msra_origin', 'resume', 'onto4', 'onto5', # 'cluener'
]

entity_count = {
    'msra_origin': 75059, # 45033
    'resume': 13438,  # in 3821
    'onto4': 13372,  # in 15724
    'onto5': 0,  #
    'weibo': 0,  #
    'cluener': 23338,  # in 10748
    'findoc': 120685,  # in 99581
    'scierc': 5598,  # in 1861
    'conll03': 23497,  # in 14040
    'ace04': 22204,  # in 6202
    'ace05': 24827,  # in 7299
}

In [4]:
n_samples = {}
n_spans = {}
n_chars = {}
n_chars_seg = {}

# from tqdm import tqdm

for dn in cn_dataset_names:
    file_path = f"{data_dir}/{dn}/train.json"
    if dn == 'scierc':
        n_entities = 0
        sentences = []
        for line in open(file_path, 'r'):
            sentences.extend(json.loads(line)['sentences'])
            n_entities += len(json.loads(line)['ner'])
        print(n_entities)
    else:
        doc = json.load(open(file_path, 'r'))
        sentences = doc['sentences']
        n_entities = len(doc['ner'])
        print(n_entities)
    
    n_samples[dn] = len(sentences)
    spans = []
    seg_length = []
    for sent in sentences:
        _n_span = 0
        streak = 0
        for c in sent:
            # if c in ['，']:
            if c in ['，', '；', '。', '？', '！']:
                if streak > 25:
                    _n_span += (streak - 25) * 25
                else:
                    _n_span += (1 + streak) * streak / 2
                seg_length.append(streak)
                streak = 0
            streak += 1
        spans.append(_n_span)
    n_spans[dn] = sum(spans) / len(spans)
    n_chars[dn] = sum(map(len, sentences)) / len(sentences)
    n_chars_seg[dn] = sum(seg_length) / len(seg_length)

from pprint import pprint
pprint([n_spans, n_chars, n_chars_seg, n_samples])
print(sum(n_spans.values())/3, sum(n_chars.values())/3, sum(n_chars_seg.values())/3)
for dn in n_spans:
    print('PHI:', dn, n_spans[dn] * n_samples[dn] / entity_count[dn] - 1)

[{'msra_origin': 367.731108298359,
  'onto4': 201.8200839481048,
  'onto5': 210.62296160276262,
  'resume': 254.62182674692488},
 {'msra_origin': 48.22054937490285,
  'onto4': 31.283579241923174,
  'onto5': 32.81938772713569,
  'resume': 32.47814708191573},
 {'msra_origin': 15.155198277408935,
  'onto4': 12.239122434536448,
  'onto5': 13.979538546559823,
  'resume': 14.894167392115408},
 {'msra_origin': 45033, 'onto4': 15724, 'onto5': 36487, 'resume': 3821}]
344.93199353205046 48.26722114195915 18.756008883540204
PHI: msra_origin 219.62690683329117
PHI: resume 71.39991070099717
PHI: onto4 236.3182022135806


KeyError: 'onto5'

In [28]:
n_samples = {}
n_spans = {}
n_chars = {}
n_chars_seg = {}

# from tqdm import tqdm

for dn in ['findoc']:
    file_path = f"{data_dir}/{dn}/train.json"
    if dn == 'findoc':
        sentences = []
        for line in open(file_path, 'r'):
            sentences.extend(json.loads(line)['sentences'])
    else:
        sentences = json.load(open(file_path, 'r'))['sentences']
    
    n_samples[dn] = len(sentences)
    spans = []
    seg_length = []
    for sent in sentences:
        _n_span = 0
        streak = 0
        for c in sent:
            # if c in ['，']:
            if c in ['，', '；', '。', '？', '！']:
                if streak > 25:
                    _n_span += (streak - 25) * 25
                else:
                    _n_span += (1 + streak) * streak / 2
                seg_length.append(streak)
                streak = 0
            else:
                streak += 1
        else:
            if streak > 25:
                _n_span += (streak-25) * 25
            else:
                _n_span += (1 + streak) * streak / 2 
            seg_length.append(streak)
            streak = 0
        spans.append(_n_span)
    n_spans[dn] = sum(spans) / len(spans)
    n_chars[dn] = sum(map(len, sentences)) / len(sentences)
    n_chars_seg[dn] = sum(seg_length) / len(seg_length)

from pprint import pprint
pprint([n_spans, n_chars, n_chars_seg, n_samples])
print(sum(n_spans.values())/1, sum(n_chars.values())/1, sum(n_chars_seg.values())/1)
for dn in n_spans:
    print('PHI:', n_spans[dn] * n_samples[dn] / entity_count[dn] - 1)

[{'findoc': 608.905172673502},
 {'findoc': 75.82712565650074},
 {'findoc': 21.1493599839996},
 {'findoc': 99581}]
608.905172673502 75.82712565650074 21.1493599839996
PHI: 501.426863321871


In [39]:
n_samples = {}
n_spans = {}
n_chars = {}
n_chars_seg = {}

for dn in en_dataset_names:
    file_path = f"{data_dir}/{dn}/train.json"
    if dn == 'scierc':
        n_entities = 0
        sentences = []
        for line in open(file_path, 'r'):
            sentences.extend(json.loads(line)['sentences'])
            for ner_case in json.loads(line)['ner']:
                n_entities += len(ner_case)
        print(n_entities)
    else:
        sentences = json.load(open(file_path, 'r'))['sentences']
    
    n_samples[dn] = len(sentences)
    spans = []
    seg_length = []
    for sent in sentences:
        _n_span = 0
        streak = 0
        for c in sent:
            if c in ['.']:
                if streak > 10:
                    _n_span += (streak-10) * 10
                else:
                    _n_span += (1 + streak) * streak / 2 
                seg_length.append(streak)
                streak = 0
            else:
                streak += 1
        else:
            if streak > 10:
                _n_span += (streak-10) * 10
            else:
                _n_span += (1 + streak) * streak / 2 
            seg_length.append(streak)
            streak = 0
        spans.append(_n_span)
    n_spans[dn] = sum(spans) / len(spans)
    n_chars[dn] = sum(map(len, sentences)) / len(sentences)
    n_chars_seg[dn] = sum(seg_length) / len(seg_length)

from pprint import pprint
pprint([n_spans, n_chars, n_chars_seg, n_samples])
print(sum(n_spans.values())/4, sum(n_chars.values())/4, sum(n_chars_seg.values())/4)
for dn in n_spans:
    print('PHI:', dn, n_spans[dn] * n_samples[dn] / entity_count[dn] - 1)

5598
[{'ace04': 119.66784908094164,
  'ace05': 98.44869160158926,
  'conll03': 63.96182336182336,
  'scierc': 136.5335840945728},
 {'ace04': 22.49742018703644,
  'ace05': 19.943553911494725,
  'conll03': 14.502635327635328,
  'scierc': 24.401934443847395},
 {'ace04': 19.32838506522717,
  'ace05': 16.320554445554446,
  'conll03': 20.586655817738,
  'scierc': 23.403130059363196},
 {'ace04': 6202, 'ace05': 7299, 'conll03': 14040, 'scierc': 1861}]
104.65298703473177 20.336385967503475 19.909681346970704
PHI: conll03 37.21866621270801
PHI: ace04 32.42550891731219
PHI: ace05 27.943368107302533
PHI: scierc 44.38924615934262


In [34]:
# n_spans
print('span_counts', 9655005 / 1881743, 9655005 / 3056188.5)

# n_chars
print('sentence_length', 18831.5 / 7350.5, 18831.5 / 7350.5)

# n_chars_seg
print('sentence_length on segments', 14.282264102846437/10.74142967584387, 14.282264102846437/19.909681346970704)

span_counts 5.130883972997375 3.159165411426684
sentence_length 2.561934562274675 2.561934562274675
sentence_length on segments 1.329642750905446 0.7173527217209587
